In [1]:
import os
import sys

os.chdir('/container/mount/point/')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import pickle as pkl

from tqdm import tqdm
from sklearn import preprocessing
from scipy.cluster.hierarchy import linkage, leaves_list

from gglasso.problem import glasso_problem
from gglasso.helper.basic_linalg import adjacency_matrix
from gglasso.helper.basic_linalg import scale_array_by_diagonal

from utils.helper import transform_features, scale_array_by_diagonal
from utils.preprocessing import add_noise_to_data

In [2]:
def plot_and_save_heatmap(data, title, filename_prefix, height, width, zmid_value=0):
    """Plot and save heatmap with specified parameters."""
    heatmap = go.Heatmap(
        z=data.values,
        x=data.columns,
        y=data.index,
        colorscale='RdBu_r',
        zmid=zmid_value
    )

    layout = go.Layout(
        title=title,
        xaxis=dict(tickangle=45, side='top'),
        yaxis=dict(autorange='reversed'),
        height=height,
        width=width
    )

    fig = go.Figure(data=[heatmap], layout=layout)

    for ext in ['png', 'svg', 'pdf']:
        fig.write_image(f"plots/{ext}/{filename_prefix}.{ext}")
    for ext in ['html']:
        fig.write_html(f"plots/{ext}/{filename_prefix}.{ext}")

In [3]:
asv = pd.read_csv("data/feature_table.tsv", index_col=0, sep='\t')
control_sample_df = pd.read_csv("data/control_samples.csv", sep=",", index_col=0, low_memory=False)
cluster_df = pd.read_csv("data/cluster_df_latent.csv", sep=",", index_col =0, low_memory=False)
covariates = pd.read_csv("data/54subset_latent.csv", sep=",", index_col='u3_16s_id', low_memory=False)

asv_ige_508_filtered = pd.read_csv("data/asv_ige_508.csv", sep=",", index_col=0, low_memory=False)
co_occurrence_matrix = pd.read_csv("data/co_occurrence_matrix.csv", sep=",", index_col=0, low_memory=False)
taxa = pd.read_csv('data/taxonomy_clean.csv', sep=',', index_col=0)
taxa["ASV"] = taxa.index

# Subset the ASV data to include only columns 508 IgE samples
asv_ige_508_all = asv[asv_ige_508_filtered.columns].copy()

### Create datasets

#### Drop ASVs which are not present in 508 samples

In [4]:
clusters = ["A", "B", "C", "D", "E", "F", "I", "H", "G"] 

cluster_asv_data = {}

# Loop through each unique cluster
for cluster in clusters:
    # Get the sample IDs that belong to the current cluster
    sample_ids = cluster_df.loc[cluster_df['cluster'] == cluster].index.astype(str)
    
    # Select the columns in asv_ige_508_all corresponding to these sample IDs
    cluster_asv = asv_ige_508_all.loc[:, asv_ige_508_all.columns.isin(sample_ids)]
    
    # Store the ASV data for the current cluster in the dictionary
    cluster_asv_data[cluster] = cluster_asv

    
healthy_asv = asv_ige_508_all[control_sample_df.index.astype(str)]
allergic_asv = asv_ige_508_all.loc[:, ~asv_ige_508_all.columns.isin(control_sample_df.index.astype(str))]

cluster_asv_data["healthy"] = healthy_asv
cluster_asv_data["allergic"] = allergic_asv
cluster_asv_data["all"] = asv_ige_508_all

In [5]:
levels = ["phylum", "class", "order", "family", "genus", "species", "ASV"]

data_dict = dict()
threshold = 0.01

for cluster, asv_table in cluster_asv_data.items():
    print(f"\n=== Cluster: {cluster} ===")

    ASV_table = asv_table.copy()
    # Initialize dictionary for the current cluster
    if cluster not in data_dict:
        data_dict[cluster] = {}

    #################### DROP ZERO FEATURES ##############################
    print(f"ASVs BEFORE dropping zero features: p = {ASV_table.shape[0]}")

    ### Frequency of bacterium across all samples
    taxa_freq = ASV_table.sum(axis=1)

    ### drop features with NO COUNTS
    non_zero_taxa = taxa_freq[taxa_freq > 0]

    ### Filter ASV table based on non-zero frequencies
    ASV_table_non_zero = ASV_table[ASV_table.index.isin(non_zero_taxa.index)]

    ### Filter columns with zero mean
    asv_samples_ids = set(ASV_table_non_zero.columns)
    means = ASV_table_non_zero.mean()
    zero_mean_cols = means[means == 0].index

    ### Drop columns with zero mean if any
    if any(zero_mean_cols):
        print(f"Zero mean features found and removed: {list(zero_mean_cols)}")
        ASV_table_non_zero = ASV_table_non_zero.drop(zero_mean_cols, axis=1)

    print(f"ASVs AFTER dropping zero features: p = {ASV_table_non_zero.shape[0]}")

    for level in levels:
        print(f"\n--- Level: {level} ---")

        ##################### AGGREGATION ##############################
        counts_plus_label = ASV_table_non_zero.join(taxa[level])
        counts = counts_plus_label.groupby(level).sum()

        ### DO THE FILTERING ON EVERY LEVEL
        counts_freq = counts.astype(bool).sum(axis=1) / counts.shape[1]
        filter_threshold = counts_freq[counts_freq > threshold]
        counts_filtered = counts[counts.index.isin(filter_threshold.index)]

        ### Apply CLR transformation
        clr_counts = transform_features(counts_filtered, transformation='clr')
        mclr_counts = transform_features(counts_filtered, transformation='mclr')
        
        ### Calculate covariance
        S0 = np.cov(clr_counts.values, bias = True)
        S = scale_array_by_diagonal(S0)
        corr_df = pd.DataFrame(S, index=clr_counts.index, columns=clr_counts.index)

        data_dict[cluster][level] = {'raw_counts': counts_filtered, 
                                     'clr_counts': clr_counts, 
                                     'mclr_counts': mclr_counts,
                                     "corr": corr_df}

        print(f"{level} count table shape: p = {counts_filtered.shape[0]}, N = {counts_filtered.shape[1]}")


=== Cluster: A ===
ASVs BEFORE dropping zero features: p = 15170
ASVs AFTER dropping zero features: p = 758

--- Level: phylum ---
phylum count table shape: p = 10, N = 12

--- Level: class ---
class count table shape: p = 15, N = 12

--- Level: order ---
order count table shape: p = 30, N = 12

--- Level: family ---
family count table shape: p = 62, N = 12

--- Level: genus ---
genus count table shape: p = 249, N = 12

--- Level: species ---
species count table shape: p = 710, N = 12

--- Level: ASV ---
ASV count table shape: p = 758, N = 12

=== Cluster: B ===
ASVs BEFORE dropping zero features: p = 15170
ASVs AFTER dropping zero features: p = 1276

--- Level: phylum ---
phylum count table shape: p = 11, N = 24

--- Level: class ---
class count table shape: p = 17, N = 24

--- Level: order ---
order count table shape: p = 43, N = 24

--- Level: family ---
family count table shape: p = 89, N = 24

--- Level: genus ---
genus count table shape: p = 369, N = 24

--- Level: species ---
s

In [6]:
# with open('data/cluster_count_dict.pkl', 'wb') as f:
#     pkl.dump(data_dict, f)
    
with open('data/cluster_count_dict.pkl', 'rb') as f:
    data_dict = pkl.load(f)

### Load solutions without IgE

In [7]:
with open('data/inv_corr_dict_only_taxa_all.pkl', 'rb') as f:
    sol_dict_taxa = pkl.load(f)

In [8]:
stacked_sol_dict = dict()
level = 'family'

for cluster in data_dict.keys():
    print(cluster)
    best_lam = sol_dict_taxa[cluster][level]['model_stats']['BEST']["lambda1"]
    
    if best_lam != 1 and best_lam != 0:
        print("\n", cluster)
        print("Best lambda:", best_lam)
        
        X = data_dict[cluster][level]['raw_counts']
        df = sol_dict_taxa[cluster][level]['inv_corr']
        
        #### Cluster the edges #### 
        count_block = df.loc[X.index, X.index]
        
        print("Cluster: {0}, level:{1}".format(cluster, level))
        print("Count Shape: {0}".format(count_block.shape))

        linkage_count_block = linkage(count_block, method='average')
        ordered_indices_count_block = leaves_list(linkage_count_block)
        res_df = count_block.iloc[ordered_indices_count_block, ordered_indices_count_block]
            
        df.index.name = None

        # Reset the index to turn the index into a column
        df_reset = df.reset_index()

#         # Melt the DataFrame to create a stacked format
        stacked_df = df_reset.melt(id_vars='index', var_name='Y', value_name='inv_corr')

        # Rename the 'index' column to 'X'
        stacked_df.rename(columns={'index': 'X'}, inplace=True)

        # Optionally drop NaN values if you don't want them in the final DataFrame
        stacked_df.dropna(subset=['inv_corr'], inplace=True)

        # Store the stacked DataFrame in the dictionary
        stacked_sol_dict[cluster] = stacked_df

A
B
C
D
E
F
I

 I
Best lambda: 0.6951927961775606
Cluster: I, level:family
Count Shape: (110, 110)
H
G
healthy

 healthy
Best lambda: 0.23357214690901223
Cluster: healthy, level:family
Count Shape: (91, 91)
allergic

 allergic
Best lambda: 0.23357214690901223
Cluster: allergic, level:family
Count Shape: (93, 93)
all

 all
Best lambda: 0.23357214690901223
Cluster: all, level:family
Count Shape: (91, 91)


In [9]:
all_stacked_df = pd.concat(stacked_sol_dict.values(), ignore_index=True)

# remove duplicate edges by keeping the biggest ones
all_stacked_df = all_stacked_df.loc[all_stacked_df.groupby(['X', 'Y'])['inv_corr'].apply(lambda x: x.abs().idxmax())]

In [10]:
symmetric_df = all_stacked_df.pivot(index='X', columns='Y', values='inv_corr')
symmetric_df = symmetric_df.fillna(0)
A = symmetric_df.copy()

A.to_csv('data/super_matrix_only_taxa_main.csv', index=True, header=True)

In [11]:
for cluster in stacked_sol_dict.keys():
    
    C = A.copy()
    B = stacked_sol_dict[cluster].pivot(index='X', columns='Y', values='inv_corr').copy()
    
    cols_to_zero = C.columns[~C.columns.isin(B.columns)]
    index_to_zero = C.index[~C.index.isin(B.index)]

    # Set values to 0 in A for those columns and indices
    C[cols_to_zero] = 0
    C.loc[index_to_zero] = 0

    for index in C.index:
        if index in C.columns:
            C.at[index, index] = 1
                
    C.to_csv(f'data/super_matrix_only_taxa_{cluster}.csv', index=True, header=True)

#### Heatmaps

In [12]:
count_block = A.loc[A.index[A.index.str.startswith('f__')], 
                    A.index[A.index.str.startswith('f__')]]

linkage_count_block = linkage(count_block, method='average')
ordered_indices_count_block = leaves_list(linkage_count_block)
main_df = count_block.iloc[ordered_indices_count_block, 
                                  ordered_indices_count_block]
main_labels = main_df.index

In [13]:
healthy = pd.read_csv("data/super_matrix_only_taxa_healthy.csv", sep=",", index_col=0, low_memory=False)
allergic = pd.read_csv("data/super_matrix_only_taxa_allergic.csv", sep=",", index_col=0, low_memory=False)

# Reorder columns and indices
healthy = healthy.reindex(index=main_labels, columns=main_labels)
allergic = allergic.reindex(index=main_labels, columns=main_labels)

# Compute difference
diff_counts = healthy - allergic
# plot_and_save_heatmap(diff_counts, 'Difference: healthy - allergic', f'rel_diff_heatmap_only_taxa_{level}')

# Binary difference heatmap
binary_diff_counts = diff_counts.astype(bool).astype(int)
# plot_and_save_heatmap(data=binary_diff_counts, 'Binary Difference: healthy - allergic', f'bin_diff_heatmap_only_taxa_{level}')

abs_diff_counts = healthy.astype(bool).astype(int) - allergic.astype(bool).astype(int)
# plot_and_save_heatmap(abs_diff_counts, 'Absolute Difference: healthy - allergic', f'abs_diff_heatmap_only_taxa_{level}')

nonzero_edges = np.triu(abs_diff_counts.astype(bool).astype(int), k=1).sum()
print(f"Number of edges: {nonzero_edges}")

Number of edges: 19


In [14]:
diff_counts.to_csv(f'data/diff_only_taxa_matrix.csv', index=True, header=True)

### Load solutions with IgE

In [15]:
with open('data/inv_corr_dict_all.pkl', 'rb') as f:
    sol_dict = pkl.load(f)

In [16]:
stacked_sol_dict = dict()
level = 'family'

for cluster in data_dict.keys():
    print(cluster)
    best_lam = sol_dict[cluster][level]['model_stats']['BEST']["lambda1"]
    
    if best_lam != 1 and best_lam != 0:
        print("\n", cluster)
        print("Best lambda:", best_lam)
        
        X = data_dict[cluster][level]['raw_counts']
        df = sol_dict[cluster][level]['inv_corr']
        
        #### Cluster the edges #### 
        count_block = df.loc[X.index, X.index]
        ige_block = df.loc[~df.index.isin(X.index), ~df.columns.isin(X.index)]
        count_ige_block = df.loc[X.index, ~df.columns.isin(X.index)]
        
        print("Cluster: {0}, level:{1}".format(cluster, level))
        print("Count Shape: {0}".format(count_block.shape))
        print("IgE Shape: {0}".format(ige_block.shape))

        linkage_count_block = linkage(count_block, method='average')
        ordered_indices_count_block = leaves_list(linkage_count_block)
        re_count_block = count_block.iloc[ordered_indices_count_block, ordered_indices_count_block]
        
        
        if cluster != "healthy":
            linkage_ige_block = linkage(ige_block, method='average')
            ordered_indices_ige_block = leaves_list(linkage_ige_block)
            re_ige_block = ige_block.iloc[ordered_indices_ige_block, ordered_indices_ige_block]

            re_count_ige_block = count_ige_block.iloc[ordered_indices_count_block, ordered_indices_ige_block]

            res = np.block([[re_count_block.values, re_count_ige_block.values],
                                [re_count_ige_block.T.values, re_ige_block.values]])

            labels = list(re_count_block.columns) + list(re_ige_block.columns)
            res_df = pd.DataFrame(res, index=labels, columns=labels)
            
        else:
            res_df = re_count_block.copy()
            
        
        #### Plot the solutions #### 
#         fig = px.imshow(-20*res_df, 
#                     color_continuous_scale='RdBu_r',  # Adjust the color scale
#                     labels={'color': 'Value'},          # Label for color bar
#                     title=f'Heatmap of {level} binary filtered data ({cluster})',
#                     zmin=-1, zmax=1, color_continuous_midpoint=0)

#         # Customize layout for better visuals (optional)
#         fig.update_layout(
#             width=1200,  # Adjust width
#             height=1200 # Adjust height
#         )

#         # # Show the plot
#         fig.show()
            
        df.index.name = None

        # Reset the index to turn the index into a column
        df_reset = df.reset_index()

#         # Melt the DataFrame to create a stacked format
        stacked_df = df_reset.melt(id_vars='index', var_name='Y', value_name='inv_corr')

        # Rename the 'index' column to 'X'
        stacked_df.rename(columns={'index': 'X'}, inplace=True)

        # Optionally drop NaN values if you don't want them in the final DataFrame
        stacked_df.dropna(subset=['inv_corr'], inplace=True)

        # Store the stacked DataFrame in the dictionary
        stacked_sol_dict[cluster] = stacked_df

A
B
C
D
E
F
I

 I
Best lambda: 0.4832930238571754
Cluster: I, level:family
Count Shape: (110, 110)
IgE Shape: (51, 51)
H

 H
Best lambda: 0.6951927961775606
Cluster: H, level:family
Count Shape: (91, 91)
IgE Shape: (48, 48)
G
healthy

 healthy
Best lambda: 0.23357214690901223
Cluster: healthy, level:family
Count Shape: (91, 91)
IgE Shape: (0, 0)
allergic

 allergic
Best lambda: 0.16237767391887217
Cluster: allergic, level:family
Count Shape: (93, 93)
IgE Shape: (104, 104)
all

 all
Best lambda: 0.11288378916846892
Cluster: all, level:family
Count Shape: (91, 91)
IgE Shape: (104, 104)


In [17]:
all_stacked_df = pd.concat(stacked_sol_dict.values(), ignore_index=True)

# remove duplicate edges by keeping the biggest ones
all_stacked_df = all_stacked_df.loc[all_stacked_df.groupby(['X', 'Y'])['inv_corr'].apply(lambda x: x.abs().idxmax())]

In [18]:
symmetric_df = all_stacked_df.pivot(index='X', columns='Y', values='inv_corr')
symmetric_df = symmetric_df.fillna(0)
A = symmetric_df.copy()

A.to_csv('data/super_matrix_main.csv', index=True, header=True)

In [19]:
for cluster in stacked_sol_dict.keys():
    
    C = A.copy()
    B = stacked_sol_dict[cluster].pivot(index='X', columns='Y', values='inv_corr').copy()
    
    cols_to_zero = C.columns[~C.columns.isin(B.columns)]
    index_to_zero = C.index[~C.index.isin(B.index)]

    # Set values to 0 in A for those columns and indices
    C[cols_to_zero] = 0
    C.loc[index_to_zero] = 0

    for index in C.index:
        if index in C.columns:
            C.at[index, index] = 1
                
    C.to_csv(f'data/super_matrix_{cluster}.csv', index=True, header=True)

#### Heatmaps

In [20]:
count_block = A.loc[A.index[A.index.str.startswith('f__')], 
                    A.index[A.index.str.startswith('f__')]]

linkage_count_block = linkage(count_block, method='average')
ordered_indices_count_block = leaves_list(linkage_count_block)
re_count_block = count_block.iloc[ordered_indices_count_block, 
                                  ordered_indices_count_block]

# Extract ige_block using co-occurrence IgE order
ige_block = A.loc[A.index[A.index.isin(co_occurrence_matrix.columns)], 
                  A.index[A.index.isin(co_occurrence_matrix.columns)]]

# Extract count_ige_block
count_ige_block = A.loc[count_block.index, ige_block.index]

# Ensure count_ige_block rows are reordered like count_block
re_count_ige_block = count_ige_block.iloc[ordered_indices_count_block, :]

# Combine index and columns for the final DataFrame
main_labels = list(re_count_block.index) + list(ige_block.index)

main_df = pd.DataFrame(
    np.block([
        [re_count_block.values, re_count_ige_block.values],
        [re_count_ige_block.T.values, ige_block.values]
    ]),
    index=main_labels,
    columns=main_labels
)

In [21]:
healthy = pd.read_csv("data/super_matrix_healthy.csv", sep=",", index_col=0, low_memory=False)
allergic = pd.read_csv("data/super_matrix_allergic.csv", sep=",", index_col=0, low_memory=False)

# Reorder columns and indices
healthy = healthy.reindex(index=main_labels, columns=main_labels)
allergic = allergic.reindex(index=main_labels, columns=main_labels)

# Extract count blocks
healthy_count_block = healthy.iloc[:-104, :-104]
allergic_count_block = allergic.iloc[:-104, :-104]

### Sort by connections between taxa and IgEs
allergic_ige = allergic.iloc[:-104, -104:].astype(bool).astype(int)
connections_order = allergic_ige.astype(bool).astype(int).sum(axis=1).sort_values(ascending=False)

allergic_ige_filtered = allergic_ige.loc[:, (allergic_ige != 0).any(axis=0)]
# Perform hierarchical clustering
linkage_ige = linkage(allergic_ige_filtered.T, method='average')
column_order = leaves_list(linkage_ige)
# Reindex columns to match the clustering order
allergic_ige_clustered = allergic_ige_filtered.iloc[:, column_order]

healthy_count_block = healthy_count_block.reindex(index=connections_order.index, columns=connections_order.index)
allergic_count_block = allergic_count_block.reindex(index=connections_order.index, columns=connections_order.index)

# Compute difference
diff_counts = healthy_count_block - allergic_count_block
plot_and_save_heatmap(diff_counts, 'Difference: healthy - allergic', f'rel_diff_heatmap_{level}',
                     height=2500, width=2500)

# Binary difference heatmap
binary_diff_counts = diff_counts.astype(bool).astype(int)
binary_diff_counts_ige = binary_diff_counts.join(allergic_ige_clustered)
plot_and_save_heatmap(data=binary_diff_counts_ige, title='Binary Difference: healthy - allergic', filename_prefix=f'bin_diff_heatmap_{level}',
                     height=2500, width=4000)

# Absolute difference heatmap
abs_diff_counts = healthy_count_block.astype(bool).astype(int) - allergic_count_block.astype(bool).astype(int)
abs_diff_counts_ige = abs_diff_counts.join(allergic_ige_clustered)
plot_and_save_heatmap(abs_diff_counts_ige, 'Absolute Difference: healthy - allergic', f'abs_diff_heatmap_{level}',
                     height=2500, width=4000)

abs_diff_counts_non_empty = abs_diff_counts.loc[
    (abs_diff_counts != 0).any(axis=1),  # Drop empty rows
    (abs_diff_counts != 0).any(axis=0)   # Drop empty columns
]
abs_diff_counts_non_empty = abs_diff_counts_non_empty.join(allergic_ige_clustered)
plot_and_save_heatmap(abs_diff_counts_non_empty, 'Absolute Difference (filtered): healthy - allergic', f'abs_diff_heatmap_{level}_filtered',
                     height=1500, width=2500)

nonzero_edges = np.triu(abs_diff_counts.astype(bool).astype(int), k=1).sum()
print(f"Number of edges: {nonzero_edges}")

Number of edges: 85


### Compare solutions

In [22]:
### only counts
main_df = pd.read_csv("data/super_matrix_only_taxa_main.csv", sep=",", index_col=0, low_memory=False)
healthy = pd.read_csv("data/super_matrix_only_taxa_healthy.csv", sep=",", index_col=0, low_memory=False)
allergic = pd.read_csv("data/super_matrix_only_taxa_allergic.csv", sep=",", index_col=0, low_memory=False)

### counts + IgEs
main_df_ige = pd.read_csv("data/super_matrix_main.csv", sep=",", index_col=0, low_memory=False)
healthy_ige = pd.read_csv("data/super_matrix_healthy.csv", sep=",", index_col=0, low_memory=False)
allergic_ige = pd.read_csv("data/super_matrix_allergic.csv", sep=",", index_col=0, low_memory=False)

In [23]:
# # Calculate the difference
healthy_count_block = healthy_ige.filter(regex="^f__", axis=0).filter(regex="^f__", axis=1)
allergic_count_block = allergic_ige.filter(regex="^f__", axis=0).filter(regex="^f__", axis=1)
### we compare presence/absence -> .astype(bool).astype(int)
diff_ige_counts = healthy_count_block.astype(bool).astype(int) - allergic_count_block.astype(bool).astype(int)

diff_ige = healthy_ige.copy()
diff_ige.update(diff_ige_counts)

diff_ige.to_csv(f'data/diff_matrix.csv', index=True, header=True)

In [24]:
diff_ige.loc['f__Veillonellaceae;'][diff_ige.loc['f__Veillonellaceae;'] != 0].index.tolist()

['f__Eubacteriaceae;',
 'f__Saccharimonadaceae;',
 'f__unknown_15;',
 'f__unknown_27;']

In [25]:
diff_counts = pd.read_csv("data/diff_only_taxa_matrix.csv", sep=",", index_col=0, low_memory=False)
diff_ige = pd.read_csv("data/diff_matrix.csv", sep=",", index_col=0, low_memory=False)

# Filter columns in diff_counts that are not full of zeros 
filtered_diff_counts = diff_counts.loc[
    (diff_counts != 0).any(axis=1),  
    (diff_counts != 0).any(axis=0)]

# Filter columns in diff_ige that are not full of zeros and start with "f__"
filtered_diff_ige = diff_ige.loc[
    (diff_ige != 0).any(axis=1) & diff_ige.columns.str.startswith("f__"),  
    (diff_ige != 0).any(axis=0) & diff_ige.columns.str.startswith("f__")]

filtered_diff_counts = filtered_diff_counts.reindex(index=filtered_diff_ige.index, 
                                                    columns=filtered_diff_ige.columns, fill_value=0)

filtered_diff_counts.to_csv(f'data/diff_matrix_filtered.csv', index=True, header=True)
filtered_diff_ige.to_csv(f'data/diff_matrix_filtered_ige.csv', index=True, header=True)

### Prepare data for NetComi

In [26]:
ige_data = pd.read_csv("data/ige_kora_subset_all.csv", sep=",", index_col ="u3_16s_id", low_memory=False)
print("IgE Shape:", ige_data.shape)

# Check for columns with only zeros
zero_columns = ige_data.columns[(ige_data == 0).all()]

# Print the columns with only zeros and their count
if not zero_columns.empty:
    print(f"Columns with only zeros: {list(zero_columns)}")
    print(f"Dropped Number of columns with only zeros: {len(zero_columns)}")
    ige_data = ige_data.drop(columns=zero_columns)
    ige_data = ige_data.astype(bool).astype(int)
    ige_data.index = ige_data.index.astype(str)

else:
    print("No columns contain only zeros.")

IgE Shape: (508, 104)
No columns contain only zeros.


In [27]:
dataframes = []

# Iterate over each cluster in data_dict
for cluster in stacked_sol_dict.keys():
    # Extract the 'raw_counts' DataFrame for the 'family' level of each cluster
    df = data_dict[cluster]['family']['raw_counts']
    dataframes.append(df)

# Concatenate all DataFrames by columns
merged_df = pd.concat(dataframes)
merged_df = merged_df.fillna(0)

In [28]:
df = merged_df.copy()

has_duplicates = df.index.duplicated().any()

print(f"Does the index have duplicates? {has_duplicates}")

### drop duplicates
df['non_zero_count'] = (df != 0).sum(axis=1) # Create a helper column that counts the number of non-zero values per row
df_sorted = df.sort_values(by='non_zero_count', ascending=False) # Sort the DataFrame by the non-zero count in descending order
df_unique = df_sorted[~df_sorted.index.duplicated(keep='first')] # Drop duplicates based on the index, keeping the row with the highest non-zero count
df_unique = df_unique.drop(columns='non_zero_count') # Drop the helper column as it's no longer needed

has_duplicates = df_unique.index.duplicated().any()

print(f"Does the index have duplicates after? {has_duplicates}")

Does the index have duplicates? True
Does the index have duplicates after? False


In [29]:
# Transform the data
mclr_counts_family = transform_features(df_unique, transformation='mclr')

# Transpose ige_data
ige_T = ige_data.T

# Ensure column types are strings for alignment
ige_T.columns = ige_T.columns.astype(str)

# Reorder ige_T to match mclr_counts_family column order
ige_T = ige_T[mclr_counts_family.columns]

# Concatenate the data
all_counts_ige = pd.concat([mclr_counts_family, ige_T])

all_counts_ige.T.to_csv('data/all_counts_ige.csv', index=True, header=True)

In [30]:
filtered_taxa = taxa[taxa['family'].isin(df_unique.index)]

# Remove duplicate entries based on the 'family' column in filtered_taxa
filtered_taxa = filtered_taxa.drop_duplicates(subset='family')

# Create an empty DataFrame with the same index as df_unique
label_df = pd.DataFrame(index=df_unique.index)

# Map the 'phylum' from filtered_taxa based on the 'family' to the new DataFrame
label_df['phylum'] = filtered_taxa.set_index('family')['phylum']

# Reindex to ensure all indices of df_unique are present, including missing values
label_df = label_df.reindex(all_counts_ige.index)

label_df.index = label_df.index.str.replace('.', '_', regex=False)

label_df = label_df.fillna("p__ige")

label_df.to_csv('data/phylum_labels.csv', index=True, header=True)

#### We use R package NetCoMi for the visualization (code is available in RMarkdown)